In [39]:
from llama_model import load_model, generate_text
import pandas as pd
from groq import Groq
import os
import json
os.environ["GROQ_API_KEY"] = "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"


In [24]:
tokenizer, model = load_model()

In [25]:
val_df = pd.read_json("val.json", lines=True)
val_df.head()

,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable
0,2hop__460946_294723,"[{'idx': 0, 'title': 'Grant's First Stand', 'p...",Who is the spouse of the Green performer?,"[{'id': 460946, 'question': 'Green >> performe...",Miquette Giraudy,[],True
1,2hop__252311_366220,"[{'idx': 0, 'title': 'SICRAL 1B', 'paragraph_t...",Who founded the company that distributed the f...,"[{'id': 252311, 'question': 'UHF >> distribute...",Mike Medavoy,[],True
2,2hop__701895_752697,"[{'idx': 0, 'title': 'Pangi Territory', 'parag...",What administrative territorial entity is the ...,"[{'id': 701895, 'question': 'Ciudad Deportiva ...",Tamaulipas,[],True
3,2hop__259228_793698,"[{'idx': 0, 'title': 'Richmond, Virginia', 'pa...",Where is Ulrich Walter's employer headquartered?,"[{'id': 259228, 'question': 'Ulrich Walter >> ...",Cologne,[],True
4,2hop__481349_302087,"[{'idx': 0, 'title': 'Market Square (High Poin...",Which company owns the manufacturer of Learjet...,"[{'id': 481349, 'question': 'Learjet 60 >> man...",Bombardier Inc.,[Bombardier],True


In [26]:
for i in range (1000):
    paragraphss = [p['paragraph_text'] for p in val_df['paragraphs'][i]]

    context_text = " ".join(paragraphss)
    if len(context_text) < 4000:
        print(f'{i} : {len(context_text)})')

803 : 3907)


In [27]:
val_df['question_decomposition'][0]

[{'id': 460946,
  'question': 'Green >> performer',
  'answer': 'Steve Hillage',
  'paragraph_support_idx': 10},
 {'id': 294723,
  'question': '#1 >> spouse',
  'answer': 'Miquette Giraudy',
  'paragraph_support_idx': 5}]

In [31]:
client = Groq()

from tqdm import tqdm

In [52]:
print('Reponse\t\tGround Truth')
outputs = []
responses = []
gts = []
for i in range(3000):
    
    with open('baseline_results.json', 'r') as f:
        add_to_json = json.load(f)
          
    if add_to_json.get(f'{i}'):
        # print(f'{i} present saar')
        continue
    
    
    paragraphs = val_df["paragraphs"][i]
    paragraphs = [p['paragraph_text'] for p in paragraphs]
    # paragraphs = [paragraphs[10], paragraphs[5]]
    # print(paragraphs)
    paragraphs = " ".join(paragraphs)
    # print(len(paragraphs))
        
    if len(paragraphs) > 4000:
        paragraphs = paragraphs[:4000]


    complex_question = val_df["question"][i]
    # print(complex_question)

    # prompt = f'''Given the following text : {paragraphs}.\n\n The answer to the following question {complex_question}, in the shortest way possible is : '''

    # response = generate_text(tokenizer, model, prompt, max_length=5000)
    # model_answer = response[len(prompt):-1].strip()
    
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "Using the CONTEXT, give the most accurate answer to the QUESTION in the format `ANSWER : (upto 5 word answer)"
            },
            {
                "role": "user",
                "content": f"CONTEXT : {paragraphs} \nQUESTION : '{complex_question}'.\nANSWER :",
            }
        ],

        # The language model which will generate the completion.
        model="llama-3.1-8b-instant",
        temperature=0.4,
        max_tokens=10,
        top_p=1,
        stop=None,
        stream=False,
    )
    
    model_answer = chat_completion.choices[0].message.content
        
    model_answer = model_answer.replace("ANSWER : ", "")
    model_answer = model_answer.replace("\n", "")
    model_answer = model_answer.replace(".", "")
    model_answer = model_answer.strip()
    
    gt = val_df["answer"][i].replace(".", "")
    


    add_to_json.update({
        i: {
            'model_answer': model_answer,
            'gt': gt
        }
    })
    
    with open('baseline_results.json', 'w') as f:
        json.dump(add_to_json, f, indent=4)
        
    print(f'{model_answer}\t\t{gt}')
    
    # outputs.append(response)
    responses.append(model_answer)
    gts.append(gt)

Reponse		Ground Truth
Southwest City, Missouri		McDonald County
Asian Man Records		Asian Man Records
Unfortunately, there is no information about the author of		Emilia Galotti
The Chapman Stick		conga
Mountrail County		Stanley
Henry Baring of Cromer		Sir Francis Baring, 1st Baronet
The Jackson Brothers		Jackson 5
Eddie Argos is the lead singer		Eddie Argos
The National Cycle Network		national cycling route network
There is no information about Peter Andreas		Johanne Luise Heiberg
The Peace Corps in Togo		Peace Corps
Culberson County, Texas		Van Horn
Corfe Mullen, Dorset		East Dorset
Erik Erikson has none		Joan Erikson
Sundazed Records		Columbia Records
Stefan Nemanja's brothers		Saint Sava
Not mentioned in the context		Pittsburgh Pirates
South Sudan and Ethiopia		Eastern Equatoria
March 14, 1826		Amherst
Not mentioned in the text		Rohana Wijeweera
I can't find any information about 'Banc		Haliburton County
Essex County, New Jersey		Hancock County
Lorenzo de' Medici		Giuliano de' Medici

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jczqxp2nf089qng21e2vba75` on tokens per minute (TPM): Limit 20000, Used 19365, Requested 1075. Please try again in 1.32s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [36]:
# save the gts
with open("baseline_outputs.txt", "w") as f:
    for response in responses:
        f.write(response + "\n")
        
with open("baseline_gts.txt", "w") as f:
    for gt in gts:
        f.write(gt + "\n")


In [ ]:
model_answer

'Canyon, Texas, USA'

In [ ]:
# compare the responses and the ground truths with different metrics in NLP

# BLEU score
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.spice.spice import Spice
from rouge import Rouge


rouge = Rouge()
cider = Cider()
spice = Spice()
exact_matches = 0
num_partial_matches = 0

bleu_scores = []
cider_scores = []
meteor_scores = []
rouge_scores = []
spice_scores = []
for i in range(len(gts)):
    # bleu_scores.append(sentence_bleu([gts[i].split()], responses[i].split()))
    # cider_scores.append(cider.compute_score(gts[i], responses[i]))
    # meteor_scores.append(meteor_score(gts[i], responses[i]))
    rouge_scores.append(rouge.get_scores(responses[i], gts[i]))
    # spice_scores.append(spice.compute_score(gts[i], responses[i]))
    # exact match
    if gts[i] == responses[i]:
        exact_matches += 1
        
    words_gt = gts[i].split()
    words_response = responses[i].split()
    
    # partial match
    if set(words_gt).intersection(set(words_response)):
        num_partial_matches += 1
     
rouge_1_f = 0
rouge_2_f = 0
rouge_l_f = 0   

for i in range(len(rouge_scores)):
    rouge_1_f+= rouge_scores[i][0]['rouge-1']['f']
    rouge_2_f+= rouge_scores[i][0]['rouge-2']['f']
    rouge_l_f+= rouge_scores[i][0]['rouge-l']['f']
    
rouge_1_f = rouge_1_f/len(rouge_scores)
rouge_2_f = rouge_2_f/len(rouge_scores)
rouge_l_f = rouge_l_f/len(rouge_scores)

print("Average ROUGE-1 F1 score: ", rouge_1_f)
print("Average ROUGE-2 F1 score: ", rouge_2_f)
print("Average ROUGE-L F1 score: ", rouge_l_f)
    
        
print("Exact matches: ", exact_matches/len(gts))
print("Partial matches: ", num_partial_matches/len(gts))

# print("BLEU scores: ", bleu_scores)
# print("CIDEr scores: ", cider_scores)
# print("METEOR scores: ", meteor_scores)
# print("ROUGE scores: ", rouge_scores)
# print("SPICE scores: ", spice_scores)

Average ROUGE-1 F1 score:  0.07499999927777778
Average ROUGE-2 F1 score:  0.024999999812500003
Average ROUGE-L F1 score:  0.07499999927777778
Exact matches:  0.0
Partial matches:  0.15
